In [12]:
from urllib import response
import requests , json, csv
import pandas as pd
import os

import plotly.graph_objs as go

import chart_studio
import chart_studio.plotly as py

In [2]:
def get_top_picks_df(base_path: str, season: str, overallLeagueID: int, top_n: int, curr_gw: int):
    players_ids_df = pd.read_csv(base_path + 'data/' + season + '/player_idlist.csv') 
    players_ids_df.rename({'id': 'player_id'}, axis=1, inplace=True)
    players_ids_df['full_name'] = players_ids_df['first_name'] + ' ' + players_ids_df['second_name']
    players_ids_df.drop('first_name', axis=1, inplace=True)
    players_ids_df.drop('second_name', axis=1, inplace=True)

    players_stats_df = pd.read_csv(base_path + 'data/' + season + '/cleaned_players.csv') 
    players_stats_df['full_name'] = players_stats_df['first_name'] + ' ' + players_stats_df['second_name']
    players_stats_df.drop('first_name', axis=1, inplace=True)
    players_stats_df.drop('second_name', axis=1, inplace=True)

    players_df = players_ids_df.merge(players_stats_df, on=['full_name'])

    picks_df = top_managers_gw_picks_df(overallLeagueID=overallLeagueID, top_n=top_n, curr_gw=curr_gw)
    
    merged = picks_df.merge(players_df, on=['player_id'])
    merged=merged.sort_values(by=['team_id', 'gw', 'position'])
    return merged


def top_managers_gw_picks_df(overallLeagueID: int, top_n: int, curr_gw: int):
    count = 0
    picks = []
    for manager in get_top_managers_from_api(overallLeagueID):
        if count >= top_n:
            break
        count +=1
        teamID = manager['entry']
        cols =  ['team_id','gw','player_id','position','multiplier']
        parsed = get_gw_picks_from_api(teamID=teamID, gw=curr_gw)
        for i in range(len(parsed['picks'])):
            try:
                currPicks = {
                    'team_id':teamID,
                    'gw':curr_gw, 
                    'player_id':parsed['picks'][i]['element'], 
                    'position':parsed['picks'][i]['position'],
                    'multiplier':parsed['picks'][i]['multiplier']
                    }
                picks.append(currPicks)
            except:
                continue
    
    return pd.DataFrame(picks, columns = cols)


def top_managers_gw_infos_df(overallLeagueID: int, top_n: int, curr_gw: int):
    count = 0
    infos = []
    for manager in get_top_managers_from_api(overallLeagueID):
        if count >= top_n:
            break
        count +=1
        teamID = manager['entry']
        cols = ['team_id','gw','points','bench','gw_rank','transfers','hits','total_points','overall_rank','team_value','chip']
        for gw in range(1,curr_gw):
            parsed = get_gw_picks_from_api(teamID=teamID, gw=gw)
            try:
                currInfo = {
                    'team_id':teamID,
                    'gw':gw, 
                    'points':parsed['entry_history']['points'], 
                    'bench':parsed['entry_history']['points_on_bench'],
                    'gw_rank':parsed['entry_history']['rank'], 
                    'transfers':parsed['entry_history']['event_transfers'],
                    'hits':parsed['entry_history']['event_transfers_cost'], 
                    'total_points':parsed['entry_history']['total_points'],
                    'overall_rank':parsed['entry_history']['overall_rank'], 
                    'team_value':int(parsed['entry_history']['value'])/10, 
                    'chip':parsed['active_chip']
                    }
                infos.append(currInfo)
            except:
                continue
    
    return pd.DataFrame(infos, columns = cols)


def top_managers_df(overallLeagueID: int, top_n: int):
    ids = []
    managers = []
    cols  = ['rank','entry','player_name','entry_name','total']
    count = 0
    for manager in get_top_managers_from_api(overallLeagueID):
        if count >= top_n:
            break
        count +=1
        currManager = {
            'rank': manager['rank'],
            'entry': manager['entry'],
            'player_name': manager['player_name'],
            'entry_name': manager['entry_name'],
            'total': manager['total']}
        managers.append(currManager)
        ids.append(manager['entry'])

    return pd.DataFrame(managers, index=ids, columns = cols)

def get_top_managers_from_api(overallLeagueID: int):
    url = "https://fantasy.premierleague.com/api/leagues-classic/"+str(overallLeagueID)+"/standings/"
    response = requests.get(url)
    data = response.text
    parsed = json.loads(data)
    return parsed['standings']['results']


def get_gw_picks_from_api(teamID: int, gw: int):
    url = "https://fantasy.premierleague.com/api/entry/"+str(teamID)+"/event/"+str(gw)+"/picks/"
    response = requests.get(url)
    data = response.text
    return json.loads(data)

In [3]:
URL = "https://fantasy.premierleague.com/api/bootstrap-static/"
DATA = requests.get(URL).json()
CURR_GW_OBJS = [x for x in DATA['events'] if x['is_current'] == True]
if len(CURR_GW_OBJS) == 0:
    CURR_GW_OBJS = DATA['events']        
CURR_GW = CURR_GW_OBJS[-1]['id']
OVERALL_LEAGUE_ID = 314
TOP_N = 10
BASE_PATH = './../scraper/'
SEASON = '2022-23'

In [4]:
df = get_top_picks_df(base_path=BASE_PATH, season=SEASON, overallLeagueID=OVERALL_LEAGUE_ID, top_n=TOP_N, curr_gw=CURR_GW)
df

,team_id,gw,player_id,position,multiplier,full_name,goals_scored,assists,total_points,minutes,...,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type
137,386896,14,152,1,1,Vicente Guaita,0,0,44,1080,...,0.0,5,218,30.2,3,0,1,3.0,45,GK
102,386896,14,306,2,1,João Cancelo,2,2,71,1006,...,179.0,10,284,63.8,6,0,2,56.3,74,DEF
11,386896,14,357,3,1,Kieran Trippier,1,3,77,1155,...,63.0,12,352,97.0,7,0,2,61.8,58,DEF
139,386896,14,199,4,1,James Tarkowski,0,0,39,1170,...,139.0,2,229,57.7,4,0,2,1.5,44,DEF
36,386896,14,369,5,1,Miguel Almirón Rejala,7,1,79,1085,...,510.0,9,250,99.2,6,0,0,23.0,56,MID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,6658168,14,210,11,1,Aleksandar Mitrović,9,0,61,1019,...,746.0,8,215,125.3,3,0,4,27.6,69,FWD
113,6658168,14,254,12,0,Danny Ward,0,0,43,1170,...,0.0,5,239,30.0,4,0,0,26.2,41,GK
59,6658168,14,318,13,0,Erling Haaland,17,3,117,920,...,840.0,18,485,162.4,5,0,0,80.9,122,FWD
123,6658168,14,295,14,0,Neco Williams,0,1,28,982,...,148.0,1,154,51.3,3,0,3,24.7,41,DEF


In [5]:
df.columns

Index(['team_id', 'gw', 'player_id', 'position', 'multiplier', 'full_name',
       'goals_scored', 'assists', 'total_points', 'minutes', 'goals_conceded',
       'creativity', 'influence', 'threat', 'bonus', 'bps', 'ict_index',
       'clean_sheets', 'red_cards', 'yellow_cards', 'selected_by_percent',
       'now_cost', 'element_type'],
      dtype='object')

In [6]:
df['picked_by'] = df['player_id'].apply(lambda x: (df['player_id'] == x).sum())
df = df[['element_type','player_id','full_name','picked_by','total_points','now_cost','multiplier']]
df = df.drop_duplicates(subset='player_id', keep="last")

In [7]:
df

,element_type,player_id,full_name,picked_by,total_points,now_cost,multiplier
139,DEF,199,James Tarkowski,1,39,44,1
140,DEF,461,Kurt Zouma,1,45,45,0
106,DEF,285,Trent Alexander-Arnold,1,36,72,1
107,DEF,10,Benjamin White,2,53,46,1
114,MID,373,Joe Willock,1,36,49,0
69,DEF,286,Joseph Gomez,1,35,46,1
80,MID,111,Leandro Trossard,1,73,69,1
81,MID,142,Mason Mount,1,49,77,1
91,GK,113,Robert Sánchez,1,45,46,0
94,DEF,109,Joël Veltman,1,37,46,0


In [8]:
df[df.element_type == 'GK'].sort_values(by="picked_by", ascending=False, inplace=False)

,element_type,player_id,full_name,picked_by,total_points,now_cost,multiplier
113,GK,254,Danny Ward,5,43,41,0
68,GK,15,Aaron Ramsdale,3,46,49,1
0,GK,376,Nick Pope,3,62,53,1
97,GK,478,José Malheiro de Sá,3,58,51,1
138,GK,152,Vicente Guaita,2,44,45,1
91,GK,113,Robert Sánchez,1,45,46,0
148,GK,455,Lukasz Fabianski,1,48,50,0
121,GK,207,Paulo Gazzaniga Farias,1,0,40,0
50,GK,548,Daniel Iversen,1,0,39,0


In [9]:
df[df.element_type == 'DEF'].sort_values(by="picked_by", ascending=False, inplace=False)

,element_type,player_id,full_name,picked_by,total_points,now_cost,multiplier
13,DEF,357,Kieran Trippier,10,77,58,1
104,DEF,306,João Cancelo,8,71,74,1
73,DEF,26,William Saliba,4,60,52,1
61,DEF,448,Ivan Perišić,4,43,55,0
117,DEF,342,Diogo Dalot Teixeira,3,49,47,1
93,DEF,484,Max Kilman,2,39,45,0
123,DEF,295,Neco Williams,2,28,41,0
107,DEF,10,Benjamin White,2,53,46,1
134,DEF,258,Timothy Castagne,2,53,46,0
3,DEF,16,Gabriel dos Santos Magalhães,1,52,51,1


In [10]:
df[df.element_type == 'MID'].sort_values(by="picked_by", ascending=False, inplace=False)

,element_type,player_id,full_name,picked_by,total_points,now_cost,multiplier
22,MID,19,Gabriel Martinelli Silva,9,68,68,1
38,MID,369,Miguel Almirón Rejala,9,79,56,1
79,MID,346,Andreas Hoelgebaum Pereira,6,53,46,1
28,MID,301,Kevin De Bruyne,5,81,124,2
127,MID,314,Phil Foden,3,66,85,1
119,MID,428,Son Heung-min,2,53,117,1
29,MID,160,Wilfried Zaha,2,52,76,1
129,MID,169,Eberechi Eze,2,52,57,1
130,MID,283,Mohamed Salah,2,65,128,2
149,MID,467,Declan Rice,1,41,48,0


In [11]:
df[df.element_type == 'FWD'].sort_values(by="picked_by", ascending=False, inplace=False)

,element_type,player_id,full_name,picked_by,total_points,now_cost,multiplier
59,FWD,318,Erling Haaland,10,117,122,0
89,FWD,427,Harry Kane,6,83,116,2
45,FWD,28,Gabriel Fernando de Jesus,6,65,80,1
47,FWD,80,Ivan Toney,3,70,74,1
84,FWD,210,Aleksandar Mitrović,3,61,69,1
120,FWD,356,Callum Wilson,1,53,74,2
124,FWD,237,Sam Greenwood,1,5,42,0


In [21]:
def plot_df(df):
    goalkeepers = df[(df['element_type']=='GKP')]
    defenders = df[(df['element_type']=='DEF')]
    midfielders = df[(df['element_type']=='MID')]
    forwards = df[(df['element_type']=='FWD')]

    trace_gkp = get_trace(goalkeepers,'Goalkeeper')
    trace_def = get_trace(defenders,'Defender')
    trace_mid = get_trace(midfielders,'Midfielder')
    trace_fwd = get_trace(forwards,'Forward')

    data = [trace_gkp,trace_def,trace_mid,trace_fwd]

    updatemenus = list([
        dict(active=0,
             pad = {'r': 0, 't': 10},
             x = 0,
             y = 1.18,
             type = 'buttons',
             font=dict(color='#404040'),
             bgcolor = 'rgba(255,255,255,100)',
             direction = 'right',
             xanchor = 'left',
             buttons=list([   
                dict(label = 'All',
                     method = 'update',
                     args = [{'visible': [True, True, True, True]}]),
                dict(label = 'Goalkeepers',
                     method = 'update',
                     args = [{'visible': [True, False, False, False]}]),
                dict(label = 'Defenders',
                     method = 'update',
                     args = [{'visible': [False, True, False, False]}]),
                dict(label = 'Midfielders',
                     method = 'update',
                     args = [{'visible': [False, False, True, False]}]),
                dict(label = 'Forwards',
                     method = 'update',
                     args = [{'visible': [False, False, False, True]}])
            ]),
        )
    ])

    layout = go.Layout(
        modebar={'bgcolor': 'rgba(0,0,0,0)'},
        hovermode = 'closest',
        showlegend=False,
        updatemenus=updatemenus, 
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=go.layout.XAxis(
            showgrid=True,
            zeroline=False,
            color='rgba(255,255,255,1)',
            showticklabels=False,
            title=go.layout.xaxis.Title(
                text='Cost',
                font=dict(
                    size=18,
                    color='white'
                )
            )
        ),
        yaxis=go.layout.YAxis(  
            showgrid=True,
            zeroline=False,
            color='rgba(255,255,255,10)',
            showticklabels=False,
            title=go.layout.yaxis.Title(
                text='Value',
                font=dict(
                    size=18,
                    color='white'
                )
            )
        )
    )
    fig = go.Figure(data=data, layout=layout)
    return fig


def get_trace(df, position):
    return go.Scatter(
        x = df['picked_by'],        
        y = df['total_points'],
        name= (position+'s'),
        text = df['full_name'],
        mode = 'markers',        
        marker=dict(color = map_position_to_color(position),
                    size = df['now_cost'], 
                    sizeref = 1, 
                    sizemode = 'area'),
        hoverlabel= dict(
            font=dict(color='#404040'),
            bordercolor='#404040',
            bgcolor='white'
        ),
        hovertemplate = "<b>%{text}</b><br><br>" +
            "Total points: %{y:.2f}</br>"+
            "Picked by: %{x:.2f}</br>"+
            "<extra></extra>")
            
            
def map_position_to_color(position):
    if position == 'Goalkeeper':
        return 'rgba(0,53,166, 0.8)'
    elif position == 'Defender':
        return 'rgba(101,255,71, 0.8)'
    elif position == 'Midfielder':
        return 'rgba(254,213,0, 0.8)'
    else:
        return 'rgba(236,0,0, 0.8)'

In [22]:
fig = plot_df(df)
#plotly.offline.iplot(fig)
chart_studio.plotly.plot(fig,filename="vpc")

'https://plotly.com/~antoniaelek/164/value-vs-cost/'